**Kernel**
- [XGBoost CV (LB .284)](https://www.kaggle.com/aharless/xgboost-cv-lb-284)

**Dataset**
- [Porto Seguro’s Safe Driver Prediction](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction)

In [18]:
MAX_ROUNDS = 400
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50

In [19]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from numba import jit
import time
import gc

numba는 NumPy 배열/함수 및 루프를 사용하는 파이썬 코드에 가장 잘 작동하는 즉석 컴파일러이다. 함수에 일련의 데코레이터를 적용함으로써 Numba는 코드를 컴파일 할 수 있다. Numba로 데코레이트된 함수를 호출하면 해당 함수는 "즉석"에서 바로 기계어 코드로 컴파일되고 원래의 기계어 코드 속도로 실행된다

In [20]:
# Compute gini
# from CPMP's kernel https://www.kaggle.com/cpmpml/extremely-fast-gini-computation

@jit
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

In [30]:
# Funcitons from olivier's kernel
# https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = -eval_gini(labels, preds)
    return [('gini', gini_score)]

def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None,
                  val_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    '''
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior
    '''
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    
    # Compute target mean
    averages = temp.groupby(by=trn_series.name)[target.name].agg(['mean', 'count'])
    
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages['count'] - min_samples_leaf) / smoothing))
    
    # Apply average function to all target data
    prior = target.mean()
    
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages['mean'] * smoothing
    averages.drop(['mean', 'count'], axis=1, inplace=True)
    
    # Apply averages to trn series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on = trn_series.name,
        how = 'left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    
    # Apply averages to val series
    ft_val_series = pd.merge(
        val_series.to_frame(val_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on = val_series.name,
        how = 'left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge does not keep the index so restore it
    ft_val_series.index = val_series.index
    
    # Apply averages to tst series
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on = tst_series.name,
        how = 'left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    
    return add_noise(ft_trn_series, noise_level), add_noise(ft_val_series, noise_level), add_noise(ft_tst_series, noise_level)

In [22]:
# Read data
train_df = pd.read_csv('data/train.csv', na_values="-1")
test_df = pd.read_csv('data/test.csv', na_values="-1")

In [23]:
train_features = [
    "ps_car_13",
    "ps_reg_03",
    "ps_ind_05_cat",
    "ps_ind_03",
    "ps_ind_15",
    "ps_reg_02",
    "ps_car_14",
    "ps_car_12",
    "ps_car_01_cat",
    "ps_car_07_cat",
    "ps_ind_17_bin",
    "ps_car_03_cat",
    "ps_reg_01",
    "ps_car_15",
    "ps_ind_01",
    "ps_ind_16_bin",
    "ps_ind_07_bin",
    "ps_car_06_cat",
    "ps_car_04_cat",
    "ps_ind_06_bin",
    "ps_car_09_cat",
    "ps_car_02_cat",
    "ps_ind_02_cat",
    "ps_car_11",
    "ps_car_05_cat",
    "ps_calc_09",
    "ps_calc_05",
    "ps_ind_08_bin",
    "ps_car_08_cat",
    "ps_ind_09_bin",
    "ps_ind_04_cat",
    "ps_ind_18_bin",
    "ps_ind_12_bin",
    "ps_ind_14"
]

combs = [
    ('ps_reg_01', 'ps_car_02_cat'),
    ('ps_reg_01', 'ps_car_04_cat')
]

In [24]:
# Process data

id_test = test_df['id'].values
id_train = train_df['id'].values
y = train_df['target']

start = time.time()
for n_c, (f1, f2) in enumerate(combs):
    name1 = f1 + "_plus_" + f2
    print('current feature %60s %4d in %5.1f' % (name1, n_c + 1, (time.time() - start) / 60), end='')
    print('\r' * 75, end='')
    
    train_df[name1] = train_df[f1].apply(lambda x: str(x)) + "_" + train_df[f2].apply(lambda x: str(x))
    test_df[name1] = test_df[f1].apply(lambda x: str(2)) + "_" + test_df[f2].apply(lambda x: str(x))
    
    # Label Encode
    lbl = LabelEncoder()
    lbl.fit(list(train_df[name1].values) + list(test_df[name1].values))
    train_df[name1] = lbl.transform(list(train_df[name1].values))
    test_df[name1] = lbl.transform(list(test_df[name1].values))
    
    train_features.append(name1)
    
X = train_df[train_features]
test_df = test_df[train_features]

f_cats = [f for f in X.columns if "_cat" in f]

current feature                                 ps_reg_01_plus_ps_car_04_cat    2 in   0.0

In [26]:
y_valid_pred = 0*y
y_test_pred = 0

In [27]:
# Set up folds

K = 5
kf = KFold(n_splits = K, random_state = 1, shuffle = True)
np.random.seed(0)

In [32]:
# Set up classifier

model = XGBClassifier(
    n_estimators = MAX_ROUNDS,
    max_depth = 4,
    eval_metric='mlogloss', # objective = "binary:logistic",
    learning_rate = LEARNING_RATE,
    subsample = .8,
    min_child_weight = 6,
    colsample_bytree = .8,
    scale_pos_weight = 1.6,
    gamma = 10,
    reg_alpha = 8,
    reg_lambda = 1.3
)

In [33]:
# Run CV

for i, (train_index, test_index) in enumerate(kf.split(train_df)):
    # Create data for this fold
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index, :].copy(), X.iloc[test_index, :].copy()
    X_test = test_df.copy()
    print("\nFold", i)
    
    # Enocode data
    for f in f_cats:
        X_train[f + "_avg"], X_valid[f + "_avg"], X_test[f + "_avg"] = target_encode(
            trn_series = X_train[f],
            val_series = X_valid[f],
            tst_series = X_test[f],
            target = y_train,
            min_samples_leaf = 200,
            smoothing = 10,
            noise_level = 0
        )
        
    # Run model for this fold
    if OPTIMIZE_ROUNDS:
        eval_set = [(X_valid, y_valid)]
        fit_model = model.fit(X_train, y_train,
                              eval_set = eval_set,
                              eval_metric = gini_xgb,
                              early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                             verbose = False)
        print('Best N trees = ', model.best_ntree_limit)
        print('Best gini = ', model.best_score)
    else:
        fit_model = model.fit(X_train, y_train)
    
    # Generate validation predictions for this fold
    pred = fit_model.predict_proba(X_valid)[:, 1]
    print("   Gini = ", eval_gini(y_valid, pred))
    y_valid_pred.iloc[test_index] = pred
    
    # Accumulate test set predictions
    y_test_pred += fit_model.predict_proba(X_test)[:, 1]
    
    del X_test, X_train, X_valid, y_train
    
y_test_pred /= K    # Average test set predictions

print("\nGini for full training set:")
eval_gini(y, y_valid_pred)


Fold 0


C:\Users\User\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\User\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


   Gini =  0.2851059728784705

Fold 1
   Gini =  0.28185495483845957

Fold 2
   Gini =  0.27429910138514

Fold 3
   Gini =  0.2991202920581566

Fold 4
   Gini =  0.2857903122299573

Gini for full training set:


0.28501477642381845

In [34]:
# Save validation predictions for stacking/ensembling

val = pd.DataFrame()
val['id'] = id_train
val['target'] = y_valid_pred.values
val.to_csv('data/xgb_valid.csv', float_format='%.6f', index=False)

In [35]:
# Create submission file

sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = y_test_pred
sub.to_csv('data/xgb_submit.csv', float_format='%.6f', index=False)